## Kommentare füge ich noch hinzu! Erstmal nur der Code.
### Auch die Reihenfolge kann sich noch ändern
#### Wichtig ist, dass alle Zellen über einer Zelle ausgeführt werden müssen, damit die Zellen darunter funktionieren!
#### Außerdem, können die lambda-Methoden eine Weile dauern

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

In [ ]:
boston_2018_data = pd.read_csv('resources/boston_2018.csv')
boston_2018_data.head(10)

In [ ]:
len(boston_2018_data)-len(boston_2018_data.dropna())

In [ ]:
boston_2018_data_clean = boston_2018_data.dropna(axis=0)
boston_2018_data_clean.head(1000000)

In [ ]:
def clean_date(x):
    return x.split('.')[0]

boston_2018_data_clean['start_time'] = boston_2018_data_clean['start_time'].apply(lambda x: clean_date(x))
boston_2018_data_clean['end_time'] = boston_2018_data_clean['end_time'].apply(lambda x: clean_date(x))
boston_2018_data_clean.head(1000000)

In [ ]:
def get_date(x):
    return datetime.strptime(x, "%Y-%m-%d %H:%M:%S").date()

def get_weekday(x):
    return datetime.strptime(x, "%Y-%m-%d %H:%M:%S").weekday()

def get_hour(x):
    return datetime.strptime(x, "%Y-%m-%d %H:%M:%S").hour

def get_month(x):
    return datetime.strptime(x, "%Y-%m-%d %H:%M:%S").month

In [ ]:
boston_2018_data_clean.reset_index(drop=True)
boston_2018_data_clean['date'] = boston_2018_data_clean['start_time'].apply(lambda x: get_date(x))

In [ ]:
boston_2018_data_clean['weekday'] = boston_2018_data_clean['start_time'].apply(lambda x: get_weekday(x))

In [ ]:
boston_2018_data_clean['hour'] = boston_2018_data_clean['start_time'].apply(lambda x: get_hour(x))

In [ ]:
boston_2018_data_clean['month'] = boston_2018_data_clean['start_time'].apply(lambda x: get_month(x))

In [ ]:
boston_2018_data_clean.head(5)

In [ ]:
bike_stations = pd.read_csv('resources/current_bluebikes_stations.csv')
bike_stations.head(5)

In [ ]:
len(bike_stations) - len(bike_stations.dropna())

In [ ]:
bike_stations.columns = bike_stations.iloc[0]
bike_stations.drop(bike_stations.index[0])

In [ ]:
boston_2018_start_geodata = pd.merge(boston_2018_data_clean, bike_stations[['Name','Latitude', 'Longitude', 'District']], left_on="start_station_name", right_on="Name", how="left")
boston_2018_start_geodata = boston_2018_start_geodata.rename(columns = {'Latitude': 'latitude_start', 'Longitude': 'longitude_start', "District": "district_start"})
boston_2018_start_geodata['start_coordinates'] = list(zip(boston_2018_start_geodata["latitude_start"],boston_2018_start_geodata["longitude_start"]))
del boston_2018_start_geodata['Name']

boston_2018_trip_geodata = pd.merge(boston_2018_start_geodata, bike_stations[['Name','Latitude', 'Longitude', 'District']], left_on="end_station_name", right_on="Name", how="left")
boston_2018_trip_geodata = boston_2018_trip_geodata.rename(columns = {'Latitude': 'latitude_end', 'Longitude': 'longitude_end', "District": "district_end"})
boston_2018_trip_geodata['end_coordinates'] = list(zip(boston_2018_trip_geodata["latitude_end"],boston_2018_trip_geodata["longitude_end"]))
del boston_2018_trip_geodata['Name']

boston_2018_trip_geodata.head(5)

In [ ]:
len(boston_2018_trip_geodata)-len(boston_2018_trip_geodata.dropna())

In [ ]:
boston_2018_trip_geodata_clean = boston_2018_trip_geodata.dropna()
len(boston_2018_trip_geodata_clean)-len(boston_2018_trip_geodata_clean.dropna())

# Adding temperature data

In [ ]:
boston_weather = pd.read_csv('resources/weather_hourly_boston.csv')
boston_weather.head(10)

In [ ]:
len(boston_weather) - len(boston_weather.dropna())

In [ ]:
boston_weather_clean = boston_weather.dropna()
len(boston_weather_clean) - len(boston_weather_clean.dropna())

In [ ]:
boston_weather_clean.reset_index(drop=True)
pd.options.mode.chained_assignment = None
boston_weather_clean['date'] = boston_weather_clean['date_time'].apply(lambda x:get_date(x))
boston_weather_clean['hour'] = boston_weather_clean['date_time'].apply(lambda x:get_hour(x))
boston_weather_clean.head(10)

In [ ]:
boston_2018_trip_geodata_with_temp = pd.merge(boston_2018_trip_geodata_clean, 
                                              boston_weather_clean[['max_temp','min_temp', 'precip', 'date', 'hour']],
                                              on=['date', 'hour'], how="left")
boston_2018_trip_geodata_with_temp.head(5)

In [ ]:
len(boston_2018_trip_geodata_with_temp)-len(boston_2018_trip_geodata_with_temp.dropna())

In [ ]:
boston_2018_trip_geodata_with_temp_clean = boston_2018_trip_geodata_with_temp.dropna()
len(boston_2018_trip_geodata_with_temp_clean)-len(boston_2018_trip_geodata_with_temp_clean.dropna())

# Geographical Data

In [ ]:
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import HeatMapWithTime

### Split data into four datasets, based on the month, and then visualize amount of start and end districts of trips for each of these datasets

In [ ]:
f, axes = plt.subplots(2, 4, sharey=True, figsize=(20, 10))

sns.countplot(x="district_start",data=boston_2018_trip_geodata_clean[boston_2018_trip_geodata_clean["month"] < 4], ax=axes[0][0],palette="OrRd")
sns.countplot(x="district_start",data=boston_2018_trip_geodata_clean[(boston_2018_trip_geodata_clean["month"] >= 4) & (boston_2018_trip_geodata_clean["month"] < 10)], ax=axes[0][1],palette="OrRd")
sns.countplot(x="district_start",data=boston_2018_trip_geodata_clean[(boston_2018_trip_geodata_clean["month"] >= 7) & (boston_2018_trip_geodata_clean["month"] < 10)], ax=axes[0][2],palette="OrRd")
sns.countplot(x="district_start",data=boston_2018_trip_geodata_clean[(boston_2018_trip_geodata_clean["month"] >= 10) & (boston_2018_trip_geodata_clean["month"] < 13)], ax=axes[0][3],palette="OrRd")

sns.countplot(x="district_end",data=boston_2018_trip_geodata_clean[boston_2018_trip_geodata_clean["month"] < 4], ax=axes[1][0],palette="OrRd")
sns.countplot(x="district_end",data=boston_2018_trip_geodata_clean[(boston_2018_trip_geodata_clean["month"] >= 4) & (boston_2018_trip_geodata_clean["month"] < 10)], ax=axes[1][1],palette="OrRd")
sns.countplot(x="district_end",data=boston_2018_trip_geodata_clean[(boston_2018_trip_geodata_clean["month"] >= 7) & (boston_2018_trip_geodata_clean["month"] < 10)], ax=axes[1][2],palette="OrRd")
sns.countplot(x="district_end",data=boston_2018_trip_geodata_clean[(boston_2018_trip_geodata_clean["month"] >= 10) & (boston_2018_trip_geodata_clean["month"] < 13)], ax=axes[1][3],palette="OrRd")

### Visualing amount of trips started in a district and amount of trips ended there

Splitting data into four, based on the time of day

In [ ]:
boston_2018_0_to_6 = boston_2018_trip_geodata_clean[boston_2018_trip_geodata_clean["hour"] < 6]
boston_2018_6_to_12 = boston_2018_trip_geodata_clean[(boston_2018_trip_geodata_clean["hour"] >= 6) & (boston_2018_trip_geodata_clean["hour"] < 12)]
boston_2018_12_to_18 = boston_2018_trip_geodata_clean[(boston_2018_trip_geodata_clean["hour"] >= 12) & (boston_2018_trip_geodata_clean["hour"] < 18)]
boston_2018_18_to_24 = boston_2018_trip_geodata_clean[(boston_2018_trip_geodata_clean["hour"] >= 18) & (boston_2018_trip_geodata_clean["hour"] < 24)]

In [ ]:
f, axes = plt.subplots(2, 4, sharey=True, figsize=(20, 10))

sns.countplot(x="district_start",data=boston_2018_0_to_6, ax=axes[0][0],palette="OrRd")
sns.countplot(x="district_start",data=boston_2018_6_to_12, ax=axes[0][1],palette="OrRd")
sns.countplot(x="district_start",data=boston_2018_12_to_18, ax=axes[0][2],palette="OrRd")
sns.countplot(x="district_start",data=boston_2018_18_to_24, ax=axes[0][3],palette="OrRd")


sns.countplot(x="district_end",data=boston_2018_0_to_6, ax=axes[1][0],palette="OrRd")
sns.countplot(x="district_end",data=boston_2018_6_to_12, ax=axes[1][1],palette="OrRd")
sns.countplot(x="district_end",data=boston_2018_12_to_18, ax=axes[1][2],palette="OrRd")
sns.countplot(x="district_end",data=boston_2018_18_to_24, ax=axes[1][3],palette="OrRd")

### Comparing number of trips started in a district with number of trips ended there and calculate the difference

In [ ]:
boston_2018_0_to_6_start_count = boston_2018_0_to_6.groupby('district_start').count()
boston_2018_6_to_12_start_count = boston_2018_6_to_12.groupby('district_start').count()
boston_2018_12_to_18_start_count = boston_2018_12_to_18.groupby('district_start').count()
boston_2018_18_to_24_start_count = boston_2018_18_to_24.groupby('district_start').count()

boston_2018_0_to_6_end_count = boston_2018_0_to_6.groupby('district_end').count()
boston_2018_6_to_12_end_count = boston_2018_6_to_12.groupby('district_end').count()
boston_2018_12_to_18_end_count = boston_2018_12_to_18.groupby('district_end').count()
boston_2018_18_to_24_end_count = boston_2018_18_to_24.groupby('district_end').count()

boston_2018_0_to_6_change_count = boston_2018_0_to_6_start_count - boston_2018_0_to_6_end_count
boston_2018_6_to_12_change_count = boston_2018_6_to_12_start_count - boston_2018_6_to_12_end_count
boston_2018_12_to_18_change_count = boston_2018_12_to_18_start_count - boston_2018_12_to_18_end_count
boston_2018_18_to_24_change_count = boston_2018_18_to_24_start_count - boston_2018_18_to_24_end_count
boston_2018_0_to_6_change_count

### Visualize the difference per district

In [ ]:
f, axes = plt.subplots(1, 4, sharey=True, figsize=(20, 10))

axes[0].set_title("Supply change between 0 and 6 am")
axes[1].set_title("Supply change between 6 and 12 am")
axes[2].set_title("Supply change between 12 am and 6 pm")
axes[3].set_title("Supply change between 6 and 12 pm")

sns.barplot(x = boston_2018_0_to_6_change_count.index, y="start_time", data=boston_2018_0_to_6_change_count, ax=axes[0],palette="OrRd")
sns.barplot(x=boston_2018_6_to_12_change_count.index, y="start_time", data=boston_2018_6_to_12_change_count, ax=axes[1],palette="OrRd")
sns.barplot(x=boston_2018_12_to_18_change_count.index, y="start_time", data=boston_2018_12_to_18_change_count, ax=axes[2],palette="OrRd")
sns.barplot(x=boston_2018_18_to_24_change_count.index, y="start_time", data=boston_2018_18_to_24_change_count, ax=axes[3],palette="OrRd")

In [ ]:
boston_2018_trip_geodata_clean["district_route"] = boston_2018_trip_geodata_clean["district_start"] + "," + boston_2018_trip_geodata_clean["district_end"]
boston_2018_trip_geodata_clean.head(5)

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(20, 10))

axes[0][0].set_title("Destinations when starting in Boston")
axes[0][1].set_title("Destinations when starting in Cambridge")
axes[1][0].set_title("Destinations when starting in Somerville")
axes[1][1].set_title("Destinations when starting in Brookline")
sns.countplot(x="district_route",data=boston_2018_trip_geodata_clean[boston_2018_trip_geodata_clean["district_start"] == "Boston"], ax=axes[0][0],palette="OrRd")
sns.countplot(x="district_route",data=boston_2018_trip_geodata_clean[boston_2018_trip_geodata_clean["district_start"] == "Cambridge"], ax=axes[0][1],palette="OrRd")
sns.countplot(x="district_route",data=boston_2018_trip_geodata_clean[boston_2018_trip_geodata_clean["district_start"] == "Somerville"], ax=axes[1][0],palette="OrRd")
sns.countplot(x="district_route",data=boston_2018_trip_geodata_clean[boston_2018_trip_geodata_clean["district_start"] == "Brookline"], ax=axes[1][1],palette="OrRd")

In [ ]:
f, axes = plt.subplots(1, 2, sharey=True, figsize=(20, 10))

axes[0].set_title("Subscriber starting districts")
axes[1].set_title("Customer starting districts")
sns.countplot(x="district_start",data=boston_2018_trip_geodata_clean[boston_2018_trip_geodata_clean["user_type"] == "Subscriber"], ax=axes[0],palette="OrRd")
sns.countplot(x="district_start",data=boston_2018_trip_geodata_clean[boston_2018_trip_geodata_clean["user_type"] == "Customer"], ax=axes[1],palette="OrRd")

### Timelapse of monthly start locations visualized in a heatmap

In [ ]:
coordinate_list = []

for i in range(12):
    heat_data2 = []
    for element in boston_2018_trip_geodata_clean[boston_2018_trip_geodata_clean["month"] == (i + 1)]["start_coordinates"]:
        heat_data2.append([float(element[0]), float(element[1])])
    coordinate_list.append(heat_data2)

In [ ]:
monthly_heat_map = folium.Map(location=(42.76507, -71.25371),  tiles='Stamen Toner', 
                       zoom_start=9, control_scale=True, max_zoom=20)

monthly_heat_map.add_child(plugins.HeatMapWithTime(data=coordinate_list, use_local_extrema=False, radius=25))
monthly_heat_map

## Duration (KPI)

In [ ]:
import numpy as np

In [ ]:
def get_datetime(x):
    return datetime.strptime(x, "%Y-%m-%d %H:%M:%S")
    
boston_2018_data_clean['datetime_start'] = boston_2018_data_clean['start_time'].apply(lambda x: get_datetime(x))
boston_2018_data_clean['datetime_end'] = boston_2018_data_clean['end_time'].apply(lambda x: get_datetime(x))

In [ ]:
boston_2018_data_clean['duration'] = boston_2018_data_clean['datetime_end'] - boston_2018_data_clean['datetime_start']
boston_2018_data_clean.head(10)

In [ ]:
def get_seconds(x):
    return x.total_seconds()

boston_2018_data_clean['seconds'] = boston_2018_data_clean['duration'].apply(lambda x: get_seconds(x))
boston_2018_data_clean.head(100)

In [ ]:
total_duration_per_trip = np.empty(24)
total_duration_per_hour = np.empty(24)
total_amount_of_trips = np.empty(24)
for i in range(24):
    total_duration_per_trip[i] = boston_2018_data_clean[boston_2018_data_clean["hour"] == i]["seconds"].median()
    total_duration_per_hour[i] = boston_2018_data_clean[boston_2018_data_clean["hour"] == i]["seconds"].sum()
    total_amount_of_trips[i] = boston_2018_data_clean[boston_2018_data_clean["hour"] == i]["seconds"].count()

In [ ]:
total_amount_of_trips

In [ ]:
duration_per_trip = np.divide(total_duration_per_trip, 60)
duration_per_trip

In [ ]:
total_duration_per_hour = np.divide(total_duration_per_hour, 60 * 60)
total_duration_per_hour

In [ ]:
duration_per_trip_data = pd.DataFrame(duration_per_trip, columns=['minutes'])
total_duration_per_hour_data = pd.DataFrame(total_duration_per_hour, columns=['hours'])
amount_of_trips_data = pd.DataFrame(total_amount_of_trips, columns=['amount'])

f, axes = plt.subplots(4, 1, figsize=(20, 20))

sns.barplot(x = total_duration_per_hour_data.index , y="hours", data=total_duration_per_hour_data, ax=axes[0],palette="OrRd")
axes[0].set_title("Total hours travelled per hour throughout the year (As a reference)")
sns.barplot(x = amount_of_trips_data.index , y="amount", data=amount_of_trips_data, ax=axes[1],palette="OrRd")
axes[1].set_title("Amount of trips per hour throughout the year (As a reference)")
sns.barplot(x = duration_per_trip_data.index , y="minutes", data=duration_per_trip_data, ax=axes[2],palette="OrRd")
axes[2].set_title("Median duration of trip per hour throughout the year (KPI)")
sns.boxplot(x = "hour", y = "seconds", data= boston_2018_data_clean, ax=axes[3], palette="OrRd", showfliers=False)
axes[3].set_title("Median duration of trip per hour throughout the year (KPI)")

# Prediction

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
date_counting = boston_2018_trip_geodata_with_temp_clean.groupby("date").count()["start_time"]
date_grouping = boston_2018_trip_geodata_with_temp_clean.groupby(["date", "hour"]).count()
#yp = yp.rename({"start_time": "amount"})
date_grouping = date_grouping.reset_index(level=[0,1])
date_grouping

In [ ]:
boston_2018_trip_geodata_with_temp_clean

In [ ]:
grouped_hourly_data = boston_2018_trip_geodata_with_temp_clean.groupby(["month", "weekday", "hour"]).median()[["max_temp", "precip"]]
grouped_hourly_data

In [ ]:
grouped_amount_data = boston_2018_trip_geodata_with_temp_clean.groupby(["month", "weekday", "hour"]).count()["start_time"]
grouped_amount_data.rename("amount", inplace=True)
grouped_amount_data

In [ ]:
grouped_data = pd.concat([grouped_hourly_data, grouped_amount_data], axis=1)
grouped_data

In [ ]:
#boston_2018_trip_geodata_with_temp_clean = boston_2018_trip_geodata_with_temp_clean.merge(date_grouping[["date", "hour", "start_time"]], how="inner", left_on=["date", "hour"], right_on=["date", "hour"]) 

In [ ]:
#boston_2018_trip_geodata_with_temp_clean.rename(columns={"start_time_x": "start_time", "start_time_y" : "amount"}, inplace=True)
#boston_2018_trip_geodata_with_temp_clean

In [ ]:
date_grouping["hour"].values

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))
# Plotting the data
ax.scatter(date_grouping["hour"].values, date_grouping["start_time"].values, marker='x',)
ax.set_xlabel("Hour")
ax.set_ylabel("Amount of bikes used")

In [ ]:
def plot_regression_poly_scikit(X,Y,d):
    
    # initialize PolynomialFeatures
    poly_reg = PolynomialFeatures (degree = d)
    
    # Polynomial transformation
    x_poly = poly_reg.fit_transform(X.reshape(-1,1))
    
    # Fitting linear regression to polynomial features
    lin_reg_Poly = LinearRegression()
    lin_reg_Poly.fit(x_poly, Y)
    model_pred = lin_reg_Poly.predict(x_poly)
    
    # Plotting the regression line and the data (we have to transform the inputs as well!)
    x_fit = np.arange(X.min(),X.max() ,1)[:, np.newaxis]
    y_pred = lin_reg_Poly.predict(poly_reg.fit_transform(x_fit.reshape(-1,1)))
    
    plt.figure(figsize = (8,6))
    plt.scatter(X,Y,marker="x", c='C2')
    ylim = plt.ylim()
    plt.plot(x_fit,y_pred, c='C1')
    plt.xlabel("Temperature (°C)")
    plt.ylabel("Demand (GW)")
    plt.xlim([X.min()-2,X.max()+2]) # leave some space before and after limits
    plt.ylim(ylim)
    print ('The R^2 for quadratic curve is: ',r2_score(Y, model_pred))
    #print(lin_reg_Poly.coef_)

In [ ]:
plot_regression_poly_scikit(date_grouping["hour"].values, date_grouping["start_time"].values, d = 10)

In [ ]:
temp_grouping = boston_2018_trip_geodata_with_temp_clean.groupby(["date","max_temp"]).count()
temp_grouping = temp_grouping.reset_index(level=[0,1])
temp_grouping

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))
# Plotting the data
ax.scatter(temp_grouping["max_temp"], temp_grouping["start_time"], marker='x',)
ax.set_xlabel("Temperature")
ax.set_ylabel("Amount of bikes used")

In [ ]:
plot_regression_poly_scikit(temp_grouping["max_temp"].values, temp_grouping["start_time"].values, d = 10)

In [ ]:
grouped_data = grouped_data.reset_index()
grouped_data

In [ ]:
lr = LinearRegression()
x_entire = grouped_data[["weekday", "hour", "month", "max_temp", "precip"]]
y_entire = grouped_data["amount"]

In [ ]:
x_entire

In [ ]:
norm = StandardScaler()
x_entire_norm = norm.fit_transform(x_entire)

In [ ]:
x_entire_norm

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_entire_norm, y_entire, test_size=0.3,random_state=40)

#lr.fit(x_train, y_train)
# Different size ?????????????????????
#x_train = x_entire.iloc[y_train_index]
#y_train = y_entire.iloc[y_train_index]
#x_test = x_entire.iloc[y_test_index]
#y_test = y_entire.iloc[y_test_index]

In [ ]:
x_train

In [ ]:
lr.fit(x_train, y_train)

In [ ]:
y_predict = lr.predict(x_test)   

mae = mean_absolute_error(y_predict, y_test)
mse = mean_squared_error(y_predict, y_test)
determination = r2_score(y_predict, y_test)
print("Mean absolute error: ", mae)
print("Mean squared error: ", mse)
print("R² = ", determination)

In [ ]:
regr = svm.SVR()
regr.fit(x_train, y_train)

In [ ]:
y_predict = regr.predict(x_test)

mae = mean_absolute_error(y_predict, y_test)
mse = mean_squared_error(y_predict, y_test)
determination = r2_score(y_predict, y_test)
print("Mean absolute error: ", mae)
print("Mean squared error: ", mse)
print("R² = ", determination)

In [ ]:
ridge_reg = Ridge(alpha = 0.01, normalize = True, solver = 'lsqr')
ridge_reg.fit(x_train, y_train)

In [ ]:
y_predict = ridge_reg.predict(x_test)

mae = mean_absolute_error(y_predict, y_test)
mse = mean_squared_error(y_predict, y_test)
determination = r2_score(y_predict, y_test)
print("Mean absolute error: ", mae)
print("Mean squared error: ", mse)
print("R² = ", determination)

In [ ]:
lasso_reg = Lasso(alpha = 1)
lasso_reg.fit(x_train, y_train)

In [ ]:
y_predict = lasso_reg.predict(x_test)

mae = mean_absolute_error(y_predict, y_test)
mse = mean_squared_error(y_predict, y_test)
determination = r2_score(y_predict, y_test)
print("Mean absolute error: ", mae)
print("Mean squared error: ", mse)
print("R² = ", determination)

In [ ]:
KNN_reg = KNeighborsRegressor(n_neighbors=9)
KNN_model = KNN_reg.fit(x_train, y_train)

In [ ]:
y_predict = KNN_model.predict(x_test)

mae = mean_absolute_error(y_predict, y_test)
mse = mean_squared_error(y_predict, y_test)
determination = r2_score(y_predict, y_test)
print("Mean absolute error: ", mae)
print("Mean squared error: ", mse)
print("R² = ", determination)

In [ ]:
Tree_reg = DecisionTreeRegressor(max_depth=5)
tree_model = Tree_reg.fit(x_train, y_train) 

# Predict
y_hat_tree = tree_model.predict(x_test)

In [ ]:
y_predict = tree_model.predict(x_test)

mae = mean_absolute_error(y_predict, y_test)
mse = mean_squared_error(y_predict, y_test)
determination = r2_score(y_predict, y_test)
print("Mean absolute error: ", mae)
print("Mean squared error: ", mse)
print("R² = ", determination)

In [ ]:
model_log = LogisticRegression(C=1e3)   # C is a regularization term, we set it to 1000 here
model_log.fit(x_train, y_train)

In [ ]:
y_predict = model_log.predict(x_test)

mae = mean_absolute_error(y_predict, y_test)
mse = mean_squared_error(y_predict, y_test)
determination = r2_score(y_predict, y_test)
print("Mean absolute error: ", mae)
print("Mean squared error: ", mse)
print("R² = ", determination)

In [ ]:
deg=2
model_svc = SVC(kernel='poly', C=100.0, degree=deg, coef0=1.0)
model_svc.fit(x_train, y_train)

In [ ]:
y_predict = model_svc.predict(x_test)

mae = mean_absolute_error(y_predict, y_test)
mse = mean_squared_error(y_predict, y_test)
determination = r2_score(y_predict, y_test)
print("Mean absolute error: ", mae)
print("Mean squared error: ", mse)
print("R² = ", determination)

In [ ]:
gamma = 1 / (len(np.array(x_train))
             * np.array(x_train).var()) # scikit-learn uses gamma = 1/(n_features*X.var()) parameterization
model_rbf = SVC(kernel='rbf', C=10.0, gamma=100000*gamma)
model_rbf.fit(x_train, y_train)

In [ ]:
y_predict = model_rbf.predict(x_test)

mae = mean_absolute_error(y_predict, y_test)
mse = mean_squared_error(y_predict, y_test)
determination = r2_score(y_predict, y_test)
print("Mean absolute error: ", mae)
print("Mean squared error: ", mse)
print("R² = ", determination)